In [1]:
import pandas as pd

In [2]:
dataframes = {
    "01": pd.read_csv("~/real_estate/data/raw/Zip_MedianValuePerSqft_AllHomes.csv", encoding='ISO-8859-1'),
    "02": pd.read_csv("~/real_estate/data/raw/Zip_PctOfHomesDecreasingInValues_AllHomes.csv", encoding='ISO-8859-1'),
    "03": pd.read_csv("~/real_estate/data/raw/Zip_PctOfHomesIncreasingInValues_AllHomes.csv", encoding='ISO-8859-1'),
    "04": pd.read_csv("~/real_estate/data/raw/Zip_Zhvi_1bedroom.csv", encoding='ISO-8859-1'),
    "05": pd.read_csv("~/real_estate/data/raw/Zip_Zhvi_2bedroom.csv", encoding='ISO-8859-1'),
    "06": pd.read_csv("~/real_estate/data/raw/Zip_Zhvi_3bedroom.csv", encoding='ISO-8859-1'),
    "07": pd.read_csv("~/real_estate/data/raw/Zip_ZriPerSqft_AllHomes.csv", encoding='ISO-8859-1'),
    "08": pd.read_csv("~/real_estate/data/raw/Zip_Zhvi_Condominum.csv", encoding='ISO-8859-1'),
    "09": pd.read_csv("~/real_estate/data/raw/Zip_Zhvi_SingleFamilyResidence.csv", encoding='ISO-8859-1'),
    "10": pd.read_csv("~/real_estate/data/raw/fred_3monthtreasurybill.csv"),
    "11": pd.read_csv("~/real_estate/data/raw/fred_average_hourly_earnings_seasonally_adjusted.csv"),
    "12": pd.read_csv("~/real_estate/data/raw/fred_fixed_mortgage_rate.csv"),
    "13": pd.read_csv("~/real_estate/data/raw/fred_total_nonfarm_payroll.csv")
}

In [3]:
zillow_variables = {
    "01": "median_sqft_value",
    "02": "percent_decreasing",
    "03": "percent_increasing",
    "04": "zhvi_1bed",
    "05": "zhvi_2bed",
    "06": "zhvi_3bed",
    "07": "zhvi_sqft_value",
    "08": "zhvi_condo",
    "09": "zhvi_singlefam"
}

fred_variables = {
    "10": "90_day_treasury_bill_rate",
    "11": "mean_hourly_earnings_adj",
    "12": "mean_fixed_mortgage_rate",
    "13": "total_nonfarm_payroll"
}

In [4]:
dataframes_melt = {}

for i in zillow_variables.keys():
    
    df_ = dataframes[i].drop(["RegionID","SizeRank","City","State","Metro","CountyName","SizeRank"],axis=1)
    dataframes_melt[i] = pd.melt(df_, id_vars=["RegionName"]).rename(columns={"variable":"date",
                                                                                "value":zillow_variables[i]})

for i in fred_variables.keys():
    
    varname = dataframes[i].columns[1]
    df_ = dataframes[i].rename(columns={"DATE":"date",varname:fred_variables[i]})
    df_["date"] = [j[:7] for j in df_["date"].values]
    df_ = df_.groupby(["date"], as_index=False).agg('mean')
    dataframes_melt[i] = df_

In [5]:
df_zillow = pd.merge(dataframes_melt["01"], dataframes_melt["02"], on=["RegionName","date"], how="outer")
for i in ["03","04","05","06","07","08","09"]:
    df_zillow = pd.merge(df_zillow, dataframes_melt[i], on=["RegionName","date"], how="outer")

In [6]:
df = pd.merge(df_zillow, dataframes_melt["10"], on=["date"], how="left")
for i in ["11","12","13"]:
    df = pd.merge(df, dataframes_melt[i], on=["date"], how="left")

In [7]:
df = df.loc[df["date"] > '2000']
df = df.dropna()

df["month"] = [i[5:] for i in df["date"].values]
df["year"] = [i[:4] for i in df["date"].values]

df = df.drop("date", axis=1)
df.index = range(len(df))

In [10]:
df.to_csv("~/real_estate/data/unsupervised_dfs/df_20181210.csv",index=False)